In [ ]:
from source.custom_cnn.pg_custom_architecture import PG_CNN as CNN

In [ ]:
c = CNN()

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
#imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [ ]:
import torch                     # for all things PyTorch
import torch.nn as nn            # for torch.nn.Module, the parent object for PyTorch models
import torch.nn.functional as F  # for the activation function
import torch.optim as optim

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(c.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(1):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = c(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            running_loss = 0.0

#tqdm

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = c(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))


In [ ]:
def calculate_conv_output_size(input_height: int, input_width: int,
    padding: int, kernel_height: int, kernel_width: int, stride):
    output_height = (input_height + 2 * padding - kernel_height) / 2 + 1
    output_width = (input_width + 2 * padding - kernel_width) / 2 + 1
    return output_height, output_width

In [ ]:
import torchvision
import torchvision.transforms as transforms


transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [ ]:
type(transform)

In [ ]:
import torch

torch.download_extract('cifar10_tiny')

In [ ]:
from source.utils.config_manager import ConfigManager

data_dir_path = ConfigManager.get_data_path() + 'cifar10-kaggle/'

In [ ]:
transform_train = torchvision.transforms.Compose([
    # Scale the image up to a square of 40 pixels in both height and width
    torchvision.transforms.Resize(40),
    # Randomly crop a square image of 40 pixels in both height and width to
    # produce a small square of 0.64 to 1 times the area of the original
    # image, and then scale it to a square of 32 pixels in both height and
    # width
    torchvision.transforms.RandomResizedCrop(32, scale=(0.64, 1.0),
    ratio=(1.0, 1.0)),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor(),
    # Standardize each channel of the image
    torchvision.transforms.Normalize([0.4914, 0.4822, 0.4465],
    [0.2023, 0.1994, 0.2010])])

In [ ]:
transform_test = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.4914, 0.4822, 0.4465],
    [0.2023, 0.1994, 0.2010])])

In [ ]:
from source.utils.data_loading import get_data

train_iter, train_valid_iter, valid_iter, test_iter = get_data(data_dir_path, transform_train, transform_test, batch_size=4)

In [ ]:
import torchvision
import torchvision.transforms as transforms


print(torchvision.transforms.transforms.)

In [1]:
from functools import partial
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
from ray import tune
from ray.tune import CLIReporter, JupyterNotebookReporter
from ray.tune.schedulers import ASHAScheduler

from source.hyperparams_tuning import HyperparameteresTunner

# transfomers
from source.transformers import TestTransformersFactory
from source.transformers import TrainTrasformersFactory


In [2]:
class Net(nn.Module):
    def __init__(self, l1=120, l2=84):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, l1)
        self.fc2 = nn.Linear(l1, l2)
        self.fc3 = nn.Linear(l2, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [3]:
gpus_per_trial = 2
max_num_epochs = 10
num_samples = 10
    
config_criterion_1 = {
    "type": tune.choice([nn.CrossEntropyLoss])
}

config_optimizer_1 = {
    "type": tune.choice([optim.SGD]),
    "lr": tune.loguniform(1e-4, 1e-1),
}

config_net_1 = {
    "type": tune.choice([Net]),
    "transfrom_train": tune.choice([TrainTrasformersFactory.get_train_transformer]),
    "transform_test": tune.choice([TestTransformersFactory.get_test_transformer]),
    "optimizer": tune.choice([config_optimizer_1]),
    "criterion": tune.choice([config_criterion_1]),
    "l1": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
    "l2": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
}

config_net_2 = {
    "type": tune.choice([Net]),
    "transfrom_train": tune.choice([TrainTrasformersFactory.get_train_transformer]),
    "transform_test": tune.choice([TestTransformersFactory.get_test_transformer]),
    "optimizer": tune.choice([config_optimizer_1]),
    "criterion": tune.choice([config_criterion_1]),
    "l1": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
    "l2": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
}

config = {
    
    "batch_size": tune.choice([2, 4, 8, 16]),
    "epochs": 10,
    "net": tune.choice([config_net_1, config_net_2])
}

# config_criterion_1 = {
#     "type": nn.CrossEntropyLoss
# }

# config_optimizer_1 = {
#     "type": optim.SGD,
#     "lr": 0.001,
# }

# config_net_1 = {
#     "type": Net,
#     "transform_train": TrainTrasformersFactory.get_train_transformer,
#     "transform_test": TestTransformersFactory.get_test_transformer,
#     "optimizer": config_optimizer_1,
#     "criterion": config_criterion_1,
#     "l1": 16,
#     "l2": 8,
# }

# config_net_2 = {
#     "type": tune.choice([Net]),
#     "transform_train": tune.choice([TrainTrasformersFactory.get_train_transformer]),
#     "transform_test": tune.choice([TestTransformersFactory.get_test_transformer]),
#     "optimizer": tune.choice([config_optimizer_1]),
#     "criterion": tune.choice([config_criterion_1]),
#     "l1": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
#     "l2": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
# }

# config = {
#     "batch_size": 4,
#     "epochs": 10,
#     "net": config_net_1
# }

scheduler = ASHAScheduler(
    metric="loss",
    mode="min",
    max_t=max_num_epochs,
    grace_period=1,
    reduction_factor=2)

reporter = JupyterNotebookReporter(max_progress_rows=20,overwrite=False,max_report_frequency  = 10,max_error_rows=5)


In [4]:
tuner = HyperparameteresTunner()

In [5]:
tuner.tune('cifar10', config, scheduler, reporter)

TuneError: ('Trials did not complete', [__train_validate_a4ce4_00000])